<a href="https://colab.research.google.com/github/rho-selynn/592-HW3/blob/Michael/592_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)

In [32]:
controlData = np.loadtxt('Robot Log/robot_log.csv', delimiter = ';', skiprows = 1, usecols = (1,2), dtype = float)
print(np.max(controlData[:,0]))
controlData[:,0] /= 15

15.0


In [33]:
from torch.utils.data import TensorDataset
ImagesTrain = torch.tensor(Images[0:799])
steerDataTrain = torch.tensor(controlData[0:799, 0])
trainSteerDataset = TensorDataset(ImagesTrain, steerDataTrain)
ImagesTest = torch.tensor(Images[800:999])
steerDataTest = torch.tensor(controlData[800:999, 0])
testSteerDataset = TensorDataset(ImagesTest, steerDataTest)
throttleDataTrain = torch.tensor(controlData[0:799, 1])
trainThrottleDataset = TensorDataset(ImagesTrain, throttleDataTrain)
throttleDataTest = torch.tensor(controlData[800:999, 1])
testThrottleDataset = TensorDataset(ImagesTest, throttleDataTest)

In [59]:
steerTrainDataLoader = torch.utils.data.DataLoader(trainSteerDataset, batch_size=128, shuffle=True)
steerTestDataLoader = torch.utils.data.DataLoader(testSteerDataset, batch_size=128, shuffle=False)
throttleTrainDataLoader = torch.utils.data.DataLoader(trainThrottleDataset, batch_size=128, shuffle=True)
throttleTestDataLoader = torch.utils.data.DataLoader(testThrottleDataset, batch_size=128, shuffle=False)

In [63]:
class LR(torch.nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        return transformed_x4

class LRthrottle(torch.nn.Module):
    def __init__(self):
        super(LRthrottle, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
        self.activation = torch.nn.ReLU()
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        #transformed_x4 = self.activation(transformed_x4)
        return transformed_x4

steerNet = LR()
steerOptimizer = torch.optim.Adam(steerNet.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
criterionThrottle = torch.nn.L1Loss()
throttleNet = LRthrottle()
throttleOptimizer = torch.optim.Adam(throttleNet.parameters(), lr=0.01)

In [55]:
#steering trainer
train_loss_history = []
test_loss_history = []
for epoch in range(5):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(steerTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        steerOptimizer.zero_grad()
        predicted_output = steerNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        steerOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(steerTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = steerNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(steerTrainDataLoader)
    test_loss = test_loss/len(steerTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

Epoch 0 finished with train loss 40828.70913355478 and test loss 21564.193359375
Epoch 1 finished with train loss 9604.695320129395 and test loss 11691.460693359375
Epoch 2 finished with train loss 4760.403032575335 and test loss 31.240230083465576
Epoch 3 finished with train loss 1348.971915108817 and test loss 228.56264877319336
Epoch 4 finished with train loss 642.7114451272147 and test loss 87.171630859375


In [64]:
#throttle trainer
train_loss_history = []
test_loss_history = []
for epoch in range(40):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(throttleTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        throttleOptimizer.zero_grad()
        predicted_output = throttleNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        throttleOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(throttleTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = throttleNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(throttleTrainDataLoader)
    test_loss = test_loss/len(throttleTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

Epoch 0 finished with train loss 12864.206116531577 and test loss 4130.6513671875
Epoch 1 finished with train loss 2630.6804673331126 and test loss 3213.756103515625
Epoch 2 finished with train loss 1285.4959279128484 and test loss 846.3625793457031
Epoch 3 finished with train loss 335.86241722106934 and test loss 381.7509307861328
Epoch 4 finished with train loss 130.51406015668596 and test loss 155.1211700439453
Epoch 5 finished with train loss 79.93956695284162 and test loss 12.759578227996826
Epoch 6 finished with train loss 39.502500125340056 and test loss 11.8258957862854
Epoch 7 finished with train loss 8.746399862425667 and test loss 18.387251377105713
Epoch 8 finished with train loss 7.254897347518376 and test loss 2.591206192970276
Epoch 9 finished with train loss 4.5824523993900845 and test loss 0.9075955599546432
Epoch 10 finished with train loss 1.3486688477652413 and test loss 2.30023193359375
Epoch 11 finished with train loss 0.9601833479745048 and test loss 0.7938519418

In [65]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)
Images = torch.tensor(Images)

for i in np.linspace(1, 999, 100).astype(int):
  predicted_output_steer = steerNet(Images[i].float())
  predicted_output_throttle = throttleNet(Images.float()[i])
  correct_steer = controlData[i,0]
  correct_throttle = controlData[i,1]
  print ("correct steer: " + str(correct_steer) + " predicted steer: " + str(predicted_output_steer))
  print ("correct throttle: " + str(correct_throttle) + " predicted throttle: " + str(predicted_output_throttle))

correct steer: 0.0 predicted steer: tensor([[0.0807]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.4014]], grad_fn=<AddmmBackward0>)
correct steer: 0.0 predicted steer: tensor([[0.0853]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.3948]], grad_fn=<AddmmBackward0>)
correct steer: -0.7830426666666667 predicted steer: tensor([[0.0913]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.3761]], grad_fn=<AddmmBackward0>)
correct steer: -1.0 predicted steer: tensor([[0.0862]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.4444]], grad_fn=<AddmmBackward0>)
correct steer: 0.6324920666666666 predicted steer: tensor([[0.0807]], grad_fn=<AddmmBackward0>)
correct throttle: 1.0 predicted throttle: tensor([[0.4014]], grad_fn=<AddmmBackward0>)
correct steer: 0.35304953333333333 predicted steer: tensor([[0.0896]], grad_fn=<AddmmBackward0>)
correct throttle: 1.0 predicted